# Actor-Critic Method

This code is an example using Advantage Actor-Critic Method (a.k.a., A2C) The code is modified from [this one](https://github.com/pytorch/examples/blob/main/reinforcement_learning/actor_critic.py). [This website](https://medium.com/towards-data-science/understanding-actor-critic-methods-931b97b6df3f) might also be useful as a future reference, although the code can be outdated. 

There are multiple Actor-Critic Methods, but here we focus on 


In [11]:
import sys
import gym
import numpy as np
import scipy.linalg

import torch
import torch.nn            as nn
import torch.optim         as optim
import torch.nn.functional as F

gamma = 0.99
eps   = np.finfo( np.float32 ).eps

# Actor-Critic Neural Network.

We merge the two networks in a single class, although we can separate these two. Since we have merged the two Actor and Critic networks, they are updated in synchrony, and they also share the same state input.
A method which asynchronously updates the two networks are called the "Asynchronous Advantage Actor-Critic Method" (a.k.a. A3C), where the details are in [this website](https://medium.com/@shagunm1210/implementing-the-a3c-algorithm-to-train-an-agent-to-play-breakout-c0b5ce3b3405).

### Actor Network
The actor network maps state to action, hence the input is the number of states of the example, $n_s$ and the output is the number of actions $n_a$.

### Critic Network
The critic network maps state to a scalar real value, which is known to be the Value function $V^{\pi}(s)$.

### Advantage Value 



In [9]:
def Gt_calc( r_arr: np.ndarray, gamma: float = 1, is_normalize: bool = True ) :
    """
        This method uses vectors for the Gt array calculation.

        Args:
            [1] r_arr: an array of rewards for each time step. 

            [2] gamma: discount ratio, valued between 0 to 1. 
                       If gamma = 1, then there is no discount applied

            [3] is_print: a boolean value which determines whether to print out the computation time or not.

        Return:
            G_t: an array of discounted (or if gamma = 1, simple sum of) rewards
    """

    # Our Goal is to generate Gt_arr as follows:
    # Gt_arr = [ R1 + r x R2 + r^2 x R3 + ... + r^(N-1) x RN, R2 + r x R3 + ... + r^(N-2) x RN, ..., RN  ]
    # For this, we have to run the following operation. 
    # [1] Generate the following N x N matrix called tmp1
    #     |  1  r  r^2  r^3  ...  r^(N-1)   |
    #     |  0  1  r    r^2  ...  r^(N-2)   |
    #     |  0  0  1    r    ...  r^(N-3)   |
    #     |            ...                  |
    #     |  0  0  0                    1   |                
    # [REF] https://stackoverflow.com/questions/28705834/fastest-way-to-compute-upper-triangular-matrix-of-geometric-series-python
    # 
    # [2] Generate the following N x N matrix called tmp2
    #     |  R1  R2  R3  R4  ...  RN   |
    #     |  R1  R2  R3  R4  ...  RN   |
    #     |  R1  R2  R3  R4  ...  RN   |
    #     |                  ...        |
    #     |  R1  R2  R3  R4  ...  RN   |
    #
    # [3] Conduct a dot product (Hadamard Operator) between tmp1 and tmp2
    #     |  R1  r x R2  r^2 x R3  ...  r^(N-1) x RN   |
    #     |   0      R2  r^1 x R3  ...  r^(N-2) x RN   |
    #     |   0       0        R3  ...  r^(N-3) x RN   |
    #     |                        ...                 |
    #     |   0       0         0  ...           RN    |
    # [4] Take the sum along axis 01
    #     |  R1 + r x R2 + r^2 x R3 + ... + r^(N-1) x RN  |
    #     |           R2 + r^1 x R3 + ... + r^(N-2) x RN  |
    #     |                      R3 + ... + r^(N-3) x RN  |
    #     |                        ...                    |
    #     |                                           RN  |

    N = len( r_arr )

    tmp1 = scipy.linalg.toeplitz( gamma ** np.arange( N ), np.zeros( N ) ).T
    tmp2 = np.tile( r_arr ,( N,1 ) ) 
    
    Gt_arr = np.sum( tmp1 * tmp2, axis = 1 )

    if is_normalize: ( Gt_arr - Gt_arr.mean( ) ) / ( Gt_arr.std( ) + eps )

    return Gt_arr

class ActorCritic( nn.Module ):
    """
        Implements both actor and critic in one model
    """
    def __init__( self, num_input, num_output, num_hidden  ):

        super( ActorCritic, self ).__init__(  )

        # Saving this for choosing the action.
        self.n_action = num_output

        # First layer : from input to hidden variable
        self.affine1 = nn.Linear( num_input, num_hidden )

        # Second Actor's layer
        self.actor_layer = nn.Linear( num_hidden, num_output )

        # Second Critic's layer
        self.critic_layer = nn.Linear( num_hidden, 1 )

        # Optimizer 
        self.optimizer = optim.Adam( self.parameters( ), lr = 0.03 )

    def forward( self, x ):
        """
            Forward of both actor and critic.

            Args:
                - Current State Vector

            Outputs:
                - [1] action_prob: The pi( . | s ) itself
                - [2] state_value: The value function V(s)
        """

        # Forward the 1st layer
        x = F.relu( self.affine1( x ) )

        # Actor: Chooses action to take from state s_t 
        action_prob = F.softmax( self.actor_layer( x ), dim = -1 )

        # Critic: evaluates being in the state s_t
        state_value = self.critic_layer( x )

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t 

        return action_prob, state_value.squeeze( )

    def get_action_and_values( self, state: np.ndarray ):
        """
            Args:
                [1] State vector

            Outputs:
                [1]   action: The action chosen from the distribution
                [2]    value: Value from forwarding the network.
                [3] log_prob: The log probability of the action distruction
        """
        state = torch.from_numpy( state ).float( )

        # Forwarding the network.
        prob_dist, value = self.forward( state )

        # The action to take (left or right)
        action = np.random.choice( self.n_action, p = prob_dist.detach( ).numpy( ) )

        # Log-probability
        log_prob = torch.log( prob_dist[ action ] )

        return action, value, log_prob

    def update_network( self, rewards, values, log_probs ):
        """
            Training code. Calculates actor and critic loss and performs backprop.

            Args:
                -   rewards: Given the trajectory, a list of rewards 
                -    values: Given the trajectory, there is a list of states, and for each state there is an associated value function
                - log_probs: Given the trajectory, we have a list of p(a|s). 
        """
        
        Gt_arr    = torch.tensor( Gt_calc( rewards, gamma, is_normalize = True ) ) 
        
        values    = torch.stack(    values, dim = 0  )
        log_probs = torch.stack( log_probs, dim = 0  )

        adv_arr   = torch.sub( Gt_arr, values )

        policy_losses = -torch.dot( log_probs , adv_arr.float( ) )
        value_losses  = F.smooth_l1_loss( values, Gt_arr )

        # reset gradients
        self.optimizer.zero_grad()

        # sum up all the values of policy_losses and value_losses
        loss = policy_losses + value_losses 

        # perform backprop
        loss.backward( )
        self.optimizer.step()


# The Advantage Actor-Critic (A2C) Method


In [10]:
# Generate the gym of Cart-and-Pole
env = gym.make( 'CartPole-v1' ) 

# The number of states and actions are +4 and +2
ns  = env.observation_space.shape[ 0 ]
na  = env.action_space.n

# Add the Adam optimizer
a2c_nn    = ActorCritic( ns, na, 128 )
is_render = False

# The number of episodes
N_eps = 3000 

num_steps   = [ ]
all_rewards = [ ]

for i in range( N_eps ):

    # reset environment and episode reward
    state = env.reset()
    ep_reward = 0

    # for each episode, only run 9999 steps so that we don't 
    rewards   = []
    log_probs = [] 
    values    = []

    for t in range( 1000 ):

        # select action from policy
        if is_render: env.render( )

        action, value, log_prob  = a2c_nn.get_action_and_values( state )

        # Run one step
        new_state, reward, done, _ = env.step( action )

        rewards.append( reward )
        values.append( value )
        log_probs.append( log_prob )

        if done: 

            a2c_nn.update_network( rewards, values, log_probs )
            sum_rewards = sum( rewards )

            num_steps.append( t )
            all_rewards.append( sum_rewards )
            
            sys.stdout.write( "episode: {}, total reward: {}, average_reward: {}, length: {}\n".format( i , np.round( sum_rewards, decimals = 3 ),  np.round( np.mean( all_rewards[ -10 : ] ), decimals = 3 ), t ) )
            break

        state = new_state

/Users/mosesnah/Documents/projects/machine-learning-tutorial/.venv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


episode: 0, total reward: 62.0, average_reward: 62.0, length: 61
episode: 1, total reward: 34.0, average_reward: 48.0, length: 33
episode: 2, total reward: 29.0, average_reward: 41.667, length: 28
episode: 3, total reward: 42.0, average_reward: 41.75, length: 41
episode: 4, total reward: 39.0, average_reward: 41.2, length: 38
episode: 5, total reward: 42.0, average_reward: 41.333, length: 41
episode: 6, total reward: 59.0, average_reward: 43.857, length: 58
episode: 7, total reward: 53.0, average_reward: 45.0, length: 52
episode: 8, total reward: 118.0, average_reward: 53.111, length: 117
episode: 9, total reward: 42.0, average_reward: 52.0, length: 41
episode: 10, total reward: 53.0, average_reward: 51.1, length: 52
episode: 11, total reward: 62.0, average_reward: 53.9, length: 61
episode: 12, total reward: 37.0, average_reward: 54.7, length: 36
episode: 13, total reward: 39.0, average_reward: 54.4, length: 38
episode: 14, total reward: 24.0, average_reward: 52.9, length: 23
episode: 

# References 

[1] [Great post](https://danieltakeshi.github.io/2018/06/28/a2c-a3c/)
[2] [Google Co-lab example](https://colab.research.google.com/github/yfletberliac/rlss-2019/blob/master/labs/DRL.01.REINFORCE%2BA2C.ipynb#scrollTo=xDifFS9I4X7A)